<a href="https://colab.research.google.com/github/arbinydv/Machine-Learning/blob/main/W3_E3_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Arbind Kumar Yadav (1690891)
Week 3 - Exercise 3 (GPU interleaved and sequential Reduction operations with shared memory)

In this exercise, we are implementing and then comparing different parallel reduction methods usin Cupy and it's associated libraries.
"""

# Import required libraries
import cupy as cp
import numpy as np
import math

# Global values for vector, thread_per_block (tpb), and blocks_per_grid(bpg)
vector_size = 128
tpb = 32
bpg = math.ceil(vector_size / tpb)

def generate_rand_input():
    return cp.random.randn(vector_size, dtype=cp.float64)

# Task 1: Interleaved Reduction Kernel
interleaved_kernel = cp.RawKernel(r'''
extern "C"
__global__ void interleaved_reduction(double* xs, int stride) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    if (index % (2 * stride) == 0) {
        xs[index] += xs[index + stride];
    }
}''', 'interleaved_reduction')

def interleaved_reduction(vec):
    """Perform interleaved reduction on GPU."""
    for i in range(int(math.log2(vector_size))):
        stride = 2**i
        interleaved_kernel((bpg,), (tpb,), (vec, np.int32(stride)))
    return vec[0]


# Task 2: Interleaved Reduction with Shared Memory
shared_kernel = cp.RawKernel(r'''
extern "C"
__global__ void interleaved_shared(double* xs, int size) {
    __shared__ double shared_data[220];
    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + threadIdx.x;

    if (gid < size)
        shared_data[tid] = xs[gid];
    __syncthreads();

    for (int stride = 1; stride < blockDim.x; stride *= 2) {
        if (tid % (2 * stride) == 0)
            shared_data[tid] += shared_data[tid + stride];
        __syncthreads();
    }

    if (tid == 0)
        xs[blockIdx.x] = shared_data[0];
}''', 'interleaved_shared')

"""interleaved reduction with a shared memory."""

def interleaved_shared_reduction(vec):
    shared_kernel((bpg,), (tpb,), (vec, np.int32(vector_size)))
    return vec[0]

# Task 3: Sequential Reduction with Shared Memory
sequential_kernel = cp.RawKernel(r'''
extern "C"
__global__ void sequential_shared(double* xs, int size) {
    __shared__ double shared_data[220];
    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + threadIdx.x;

    if (gid < size)
        shared_data[tid] = xs[gid];
    else
        shared_data[tid] = 0.0;
    __syncthreads();

    for (int stride = 1; stride < blockDim.x; stride *= 2) {
        if (tid % (2 * stride) == 0)
            shared_data[tid] += shared_data[tid + stride];
        __syncthreads();
    }

    if (tid == 0)
        xs[blockIdx.x] = shared_data[0];
}''', 'sequential_shared')

""" sequential reduction using shared memory."""
def sequential_shared_reduction(vec):
    sequential_kernel((bpg,), (tpb,), (vec, np.int32(vector_size)))
    return vec[0]


# Generates random input vector
input_vector = generate_rand_input()

gpu_interleaved = interleaved_reduction(input_vector.copy())
gpu_shared = interleaved_shared_reduction(input_vector.copy())
gpu_sequential = sequential_shared_reduction(input_vector.copy())

cpu_sum = cp.sum(input_vector)

# Checks the results
print("T1 (Interleaved):", np.allclose(gpu_interleaved, cpu_sum, atol=1e-6))
print("T2 (Shared Memory):", np.allclose(gpu_shared, cpu_sum, atol=1e-6))
print("T3 (Sequential Shared):", np.allclose(gpu_sequential, cpu_sum, atol=1e-6))


T1 (Interleaved): True
T2 (Shared Memory): False
T3 (Sequential Shared): False
